### Import Library

In [ ]:

import pandas as pd
import sqlalchemy as sa
import json
from datetime import datetime

from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service

from sqlalchemy import MetaData, Table, Column, String, DateTime, Text, Float

from creds import creds

### Establish Connection to Database

In [ ]:
dest_engine = sa.create_engine(f"postgresql+psycopg2://{creds['user']}:{creds['pass']}@{creds['host']}:{creds['port']}/{creds['db']}"

### Drop all tables

In [ ]:
dest_engine.connect().execute('DROP SCHEMA public CASCADE; CREATE SCHEMA public;')

### Create Table

In [ ]:
def createStockListTable():
    engine = dest_engine
    metadata = MetaData(bind=engine)
    stocks = Table('saham', metadata,
                       Column('Code', String, primary_key=True),
                       Column('Name', Text,
                              nullable=False, unique=True),
                       Column('ListingDate', DateTime),
                       Column('Shares',Float ),
                       Column('ListingBoard', String),
                       Column('Sector', String),
                       )
    stocks.drop(checkfirst=True)
    stocks.create()

### Extract Data from Source

In [ ]:
def getstocklistData():
    DaftarSaham = pd.DataFrame()
    
    URL = 'https://www.idx.co.id/primary/StockData/GetSecuritiesStock?start=0&length=9999&code=&sector=Energy&board=&language=id-id'
        
    data = get_scrap_data(URL)

    # Mengubah json ke dalam bentuk DataFrame
    df = pd.DataFrame(data['data'])

    # Mengubah format tanggal menjadi datetime
    df['ListingDate'] = [datetime.strptime(
        x[:10], '%Y-%m-%d') for x in df['ListingDate']]
    df['Sector'] = "Energy"

    DaftarSaham = pd.concat([DaftarSaham, df], ignore_index=True)

    # urut berdasarkan kode saham
    DaftarSaham = DaftarSaham.sort_values(by='Code').reset_index(drop=True)

    
    return DaftarSaham

### Load Data into Database

In [ ]:
def insertStockListData():
    data = getstocklistData()
    
    with dest_engine.connect() as conn:
        data.to_sql('saham', conn, if_exists='append', index=False)